# Логистическая регрессия — Notebook для `Doctor.xlsx`

Задачи:
- Описательная статистика (`summary`), создание категориальных факторов при необходимости;
- Базовая логит-модель (`glm` с биномиальной семьёй), уравнение регрессии;
- Тесты значимости коэффициентов (Wald / z), тест значимости модели (Likelihood Ratio и Wald);
- Доверительные интервалы (confint и вручную), сравнение Logit vs Probit;
- Confusion matrix (порог 0.5), sensitivity / specificity, поиск оптимального порога (Youden index);
- (Опционально) разбиение на train/test, ROC-кривая и AUC;
- Stepwise selection по AIC (stepAIC аналог).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix

plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 11
sns.set_theme(style='whitegrid')


## 1. Загрузка данных и предобработка

In [ ]:
doctorDataFrame = pd.read_excel('../datasets/Doctor.xlsx')
display(doctorDataFrame.head())

def toFloatFromCommaSafe(x):
    try:
        if pd.isna(x):
            return np.nan
        s = str(x).strip().replace(',', '.')
        return float(s)
    except:
        return np.nan

for col in doctorDataFrame.columns:
    if col != 'y':
        doctorDataFrame[col] = doctorDataFrame[col].apply(toFloatFromCommaSafe)

doctorDataFrame['y'] = doctorDataFrame['y'].astype(int)

print('Размер данных:', doctorDataFrame.shape)
doctorDataFrame.info()

## 2. Описательная статистика (`summary`)

In [ ]:
display(doctorDataFrame.describe(include='all'))

print('\nCounts of y:')
print(doctorDataFrame['y'].value_counts())


## 3. Создание категориальной переменной (factor)

Если какая-то переменная (например, x4) должна быть фактором (категориальной), конвертируем её в `category` и при необходимости создаём дамми-переменные.


In [ ]:
doctorDataFrame['x4_cat'] = doctorDataFrame['x4'].astype('category')
print('x4 unique categories:', doctorDataFrame['x4_cat'].cat.categories)
display(doctorDataFrame[['x4','x4_cat']].head())

## 4. Базовая логит-модель (glm с биномиальной семьёй)

Модель с максимально возможным количеством предикторов (включая категориальные через C()).

In [ ]:
# Сформируем формулу: y ~ x1 + x2 + x3 + x5 + x4_cat
predictorTerms = []
for col in ['x1','x2','x3','x5']:
    if col in doctorDataFrame.columns:
        predictorTerms.append(col)
predictorTerms.append('C(x4_cat)')

formula = 'y ~ ' + ' + '.join(predictorTerms)
print('Formula:', formula)

# Fit GLM (Binomial = logistic)
glmLogitModel = smf.glm(formula=formula, data=doctorDataFrame, family=sm.families.Binomial()).fit()
print('--- GLM (Logit) summary ---')
print(glmLogitModel.summary())

## 5. Уравнение бинарной регрессии (логит)
Запишем в явном виде logit(p) = beta0 + beta1*x1 + ... и покажем коэффициенты.

In [ ]:
coefficientsLogit = glmLogitModel.params
print('Coefficients (logit):')
display(coefficientsLogit)

interceptLogit = coefficientsLogit.get('Intercept', coefficientsLogit.get('const', None))
termsLogit = []
for name, coef in coefficientsLogit.items():
    termsLogit.append(f'({coef:.4f})*{name}')
equationLogit = 'logit(p) = ' + ' + '.join(termsLogit)
print('\nRegression equation (logit scale):')
print(equationLogit)
print('\nProbability form: p = 1 / (1 + exp(-linear_predictor))')


## 6. Тест значимости коэффициентов по отдельности (Wald / z-test)

Для GLM стандартный вывод содержит `z` (Wald) и p-values; выведем статистики явно.

In [ ]:
coefEstimate = glmLogitModel.params
coefStdErr = glmLogitModel.bse
zValues = coefEstimate / coefStdErr
pValues = 2 * (1 - sps.norm.cdf(np.abs(zValues)))

resultTable = pd.DataFrame({
    'coef': coefEstimate,
    'stdErr': coefStdErr,
    'z': zValues,
    'pValue': pValues
})
print('Wald z-test for coefficients:')
display(resultTable)

In [ ]:
coeffIndex = glmLogitModel.params.index.tolist()
numCoeffs = len(coeffIndex)

waldResults = []
for i, coefName in enumerate(coeffIndex):
    # R матрица (1 x k): тест H0: beta_i = 0
    R = np.zeros((1, numCoeffs))
    R[0, i] = 1.0

    waldRes = glmLogitModel.wald_test(R, scalar=True)
    statRaw = waldRes.statistic
    waldStat = float(np.atleast_1d(statRaw).squeeze())
    waldPvalue = float(waldRes.pvalue)
    

    waldResults.append({
        'term': coefName,
        'waldStat_chi2': waldStat,
        'df': 1,
        'pValue': waldPvalue
    })

waldResultsDf = pd.DataFrame(waldResults).set_index('term')
print('Wald test (each coefficient separately):')
display(waldResultsDf)


## 7. Значимость регрессии в целом

Проверим модель в целом двумя способами:
- Wald test (в статистическом summary есть);  
- Likelihood Ratio Test: сравним модель с нулевой моделью (intercept-only).


In [ ]:
# all coefficients = 0 (включая intercept обычно; при желании можно исключить const)
allR = np.eye(numCoeffs)
waldResAll = glmLogitModel.wald_test(allR, scalar=True)

statAllRaw = waldResAll.statistic
waldStatAll = float(np.atleast_1d(statAllRaw).squeeze())
dfAll = allR.shape[0]
waldPvalueAll = float(waldResAll.pvalue)
print('Wald test (all coefficients simultaneously):')
print(f'Chi2 stat = {waldStatAll:.6f}, df = {dfAll}, p-value = {waldPvalueAll:.6g}')


# Wald statistic (в statsmodels есть поле 'deviance' и 'null_deviance') — можно посмотреть deviance reduction
nullDeviance = glmLogitModel.null_deviance
modelDeviance = glmLogitModel.deviance
devianceDiff = nullDeviance - modelDeviance
dfDiff = glmLogitModel.df_model
pValueDeviance = 1 - sps.chi2.cdf(devianceDiff, dfDiff)
print('\nDeviance null =', nullDeviance)
print('Deviance model =', modelDeviance)
print('Deviance reduction =', devianceDiff, ', df =', dfDiff, ', p-value =', pValueDeviance)

# LR test explicitly: compare log-likelihoods with intercept-only model
yVec = doctorDataFrame['y']
XIntercept = sm.add_constant(pd.DataFrame({'const': np.ones(len(doctorDataFrame))}))
nullModel = sm.GLM(yVec, XIntercept, family=sm.families.Binomial()).fit()
llfReduced = nullModel.llf
llfFull = glmLogitModel.llf
lrStat = 2 * (llfFull - llfReduced)
lrDf = glmLogitModel.df_model
lrPvalue = 1 - sps.chi2.cdf(lrStat, lrDf)
print(f'\nLLF full  = {llfFull:.6f}')
print(f'LLF null  = {llfReduced:.6f}')
print(f'LR stat   = {lrStat:.6f}')
print(f'df diff   = {dfDiff}')
print(f'LR p-value= {lrPvalue:.6e}')

## 8. Доверительные интервалы для коэффициентов регрессии

- `confint()` — стандартные (Wald) интервалы из statsmodels;  
- `confint_default` — построим вручную как coef ± z*se (обычно совпадает с confint для GLM).

In [ ]:
ciWald = glmLogitModel.conf_int(alpha=0.05)
ciWald.columns = ['ciLower','ciUpper']
ciManualLower = coefEstimate - 1.96 * coefStdErr
ciManualUpper = coefEstimate + 1.96 * coefStdErr
ciManual = pd.DataFrame({'ciManualLower': ciManualLower, 'ciManualUpper': ciManualUpper})
print('confint (Wald) from model:')
display(ciWald)
print('\nManual approx (coef ± 1.96*se):')
display(ciManual)


## 9. Сравнение Logit vs Probit

Построим Probit (GLM с link=probit) и сравним AIC / логарифмы правдоподобия / прогнозы.

In [ ]:
glmProbitModel = smf.glm(formula=formula, data=doctorDataFrame, family=sm.families.Binomial(link=sm.families.links.Probit())).fit()
print('--- Probit summary ---')
print(glmProbitModel.summary())

print('\nComparison:')
print('Logit AIC =', glmLogitModel.aic, ', Probit AIC =', glmProbitModel.aic)
print('Logit llf =', glmLogitModel.llf, ', Probit llf =', glmProbitModel.llf)


## 10. Confusion matrix при пороге 0.5, sensitivity и specificity

Напишем вспомогательные функции для матрицы и метрик.

In [ ]:
def confusionMatrixFromPreds(yTrue, probPred, threshold=0.5):
    yPred = (probPred >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(yTrue, yPred).ravel()
    cmDf = pd.DataFrame([[tn, fp],[fn, tp]], index=['Actual0','Actual1'], columns=['Pred0','Pred1'])
    sensitivity = tp / (tp + fn) if (tp + fn)>0 else np.nan
    specificity = tn / (tn + fp) if (tn + fp)>0 else np.nan
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return cmDf, sensitivity, specificity, accuracy

probPredLogit = glmLogitModel.predict(doctorDataFrame)

cm50, sens50, spec50, acc50 = confusionMatrixFromPreds(doctorDataFrame['y'].values, probPredLogit, threshold=0.5)
print('Confusion matrix (threshold=0.5):')
display(cm50)
print('\nSensitivity =', sens50, ', Specificity =', spec50, ', Accuracy =', acc50)

## 11. Поиск оптимального порога (Youden index) и матрица для него

Оптимальный порог — максимизирует (sensitivity + specificity - 1).

In [ ]:
def findOptimalCutoff(yTrue, probPred):
    fpr, tpr, thresholds = roc_curve(yTrue, probPred)
    youdenIndex = tpr - fpr
    bestIdx = np.argmax(youdenIndex)
    bestThreshold = thresholds[bestIdx]
    bestSens = tpr[bestIdx]
    bestSpec = 1 - fpr[bestIdx]
    return bestThreshold, bestSens, bestSpec, bestIdx

bestThr, bestSens, bestSpec, bestIdx = findOptimalCutoff(doctorDataFrame['y'].values, probPredLogit)
print('Optimal threshold (Youden) =', bestThr)
print('Sensitivity =', bestSens, ', Specificity =', bestSpec)

cmOptimal, sensOpt, specOpt, accOpt = confusionMatrixFromPreds(doctorDataFrame['y'].values, probPredLogit, threshold=bestThr)
print('\nConfusion matrix (optimal threshold):')
display(cmOptimal)
print('\nAccuracy =', accOpt)

## 12. Разбиение выборки на train/test (опционально) и ROC кривая

Если выборка большая, полезно оценивать модель на тестовой выборке. Здесь приведён пример с fraction = 0.7.

In [ ]:
trainFrac = 0.7
doctorTrain = doctorDataFrame.sample(frac=trainFrac, random_state=42)
doctorTest = doctorDataFrame.drop(doctorTrain.index)
print('Train size =', len(doctorTrain), ', Test size =', len(doctorTest))

glmLogitTrain = smf.glm(formula=formula, data=doctorTrain, family=sm.families.Binomial()).fit()
probPredTest = glmLogitTrain.predict(doctorTest)
fpr, tpr, thresholds = roc_curve(doctorTest['y'], probPredTest)
aucScore = roc_auc_score(doctorTest['y'], probPredTest)
print('Test AUC =', aucScore)
plt.figure()
plt.plot(fpr, tpr, label=f'AUC = {aucScore:.3f}')
plt.plot([0,1],[0,1],'--', color='grey')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve (test)')
plt.legend()
plt.show()


## 13. Stepwise selection по AIC (stepAIC аналог)

Реализуем простую stepwise selection для GLM (добавление/удаление по AIC).

In [ ]:
def stepwiseSelectionGLM(dataFrame, response, candidatePredictors, family=sm.families.Binomial(), verbose=True):
    included = []
    while True:
        changed = False
        excluded = [p for p in candidatePredictors if p not in included]
        bestAic = None
        bestToAdd = None

        for newCol in excluded:
            tryPredictors = included + [newCol]
            formulaTry = response + ' ~ ' + ' + '.join(tryPredictors) if tryPredictors else response + ' ~ 1'
            modelTry = smf.glm(formula=formulaTry, data=dataFrame, family=family).fit()
            if bestAic is None or modelTry.aic < bestAic:
                bestAic = modelTry.aic
                bestToAdd = newCol
        
        if bestToAdd is not None:
            formulaCurrent = response + ' ~ ' + ' + '.join(included) if included else response + ' ~ 1'
            modelCurrent = smf.glm(formula=formulaCurrent, data=dataFrame, family=family).fit()
            if bestAic + 1e-8 < modelCurrent.aic:
                included.append(bestToAdd)
                changed = True
                if verbose:
                    print('Add', bestToAdd, 'AIC=', bestAic)

        if included:
            bestAic = None
            worstToRemove = None
            
            for col in included:
                tryPredictors = [c for c in included if c != col]
                formulaTry = response + ' ~ ' + ' + '.join(tryPredictors) if tryPredictors else response + ' ~ 1'
                modelTry = smf.glm(formula=formulaTry, data=dataFrame, family=family).fit()
                if bestAic is None or modelTry.aic < bestAic:
                    bestAic = modelTry.aic
                    worstToRemove = col
            
            modelCurrent = smf.glm(formula=response + ' ~ ' + ' + '.join(included), data=dataFrame, family=family).fit()
            if bestAic + 1e-8 < modelCurrent.aic:
                included.remove(worstToRemove)
                changed = True
                if verbose:
                    print('Remove', worstToRemove, 'improve AIC to', bestAic)
        
        if not changed:
            break
    
    return included

candidatePredictors = []
for term in predictorTerms:
    candidatePredictors.append(term)

selectedPredictors = stepwiseSelectionGLM(doctorDataFrame, 'y', candidatePredictors, family=sm.families.Binomial(), verbose=True)
print('\nSelected predictors by stepwise AIC:', selectedPredictors)


## 14. Повторный анализ (если stepwise дал другую модель)

Если `selectedPredictors` отличается от исходных, подгоняем новую GLM и повторяем выводы/diagnostics (как выше): коэффициенты, CI, confusionMatrix, ROC и тесты.


In [ ]:
if set(selectedPredictors) != set(predictorTerms):
    formulaImproved = 'y ~ ' + ' + '.join(selectedPredictors) if selectedPredictors else 'y ~ 1'
    print('Fitting improved model formula:', formulaImproved)
    glmImproved = smf.glm(formula=formulaImproved, data=doctorDataFrame, family=sm.families.Binomial()).fit()
    print(glmImproved.summary())
else:
    print('Stepwise did not change predictors — оставляем базовую модель.')


## 15. VIF (мультиколлинеарность) — для числовых предикторов


In [ ]:
vifDf = pd.DataFrame()
numericPredictors = [c for c in ['x1','x2','x3','x5'] if c in doctorDataFrame.columns]

dummiesX4 = pd.get_dummies(doctorDataFrame['x4_cat'], prefix='x4')
vifX = pd.concat([doctorDataFrame[numericPredictors].reset_index(drop=True), dummiesX4.reset_index(drop=True)], axis=1)

vifX = vifX.dropna()
vifDf['feature'] = vifX.columns
vifDf['VIF'] = [variance_inflation_factor(vifX.values, i) for i in range(vifX.shape[1])]
print('VIF:')
display(vifDf)
